In [ ]:
from collections import OrderedDict
import pandas as pd
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

In [ ]:
model = SentenceTransformer('cl-tohoku/bert-base-japanese-whole-word-masking')

In [ ]:
es = Elasticsearch('https://localhost:9200',
                   basic_auth=('elastic', 'elastic'),
                   verify_certs=False,
                   timeout=60,
                   max_retries=5)
index = 'wiki_index'

In [ ]:
def vector_search(query):

    query_vector = list(model.encode(query, normalize_embeddings=True))
    response = es.search(
        index=index,
        size=10,
        source_includes=['title'],
        knn={
            'field': 'vector',
            'query_vector': query_vector,
            'k': 100,
            'num_candidates': 1000
        }
    )

    return response


def keyword_search(query):

    response = es.search(
        index=index,
        size=10,
        query={
            'match': {'abstract': query}
        }
    )

    return response


def hybrid_search(query):

    query_vector = list(model.encode(query, normalize_embeddings=True))
    response = es.search(
        index=index,
        size=10,
        source_includes=['title'],
        query={'match': {'abstract': query}},
        knn={
            'field': 'vector',
            'query_vector': query_vector,
            'k': 3,
            'num_candidates': 10
        }
    )

    return response


def format_response(response):

    return pd.DataFrame([
        OrderedDict({'title': row['_source']['title'],
                     'score': row['_score']
                     }) for _, row in pd.DataFrame(response['hits']['hits']).iterrows()])

In [ ]:
query = 'コンピュータ'
response = keyword_search(query)
print(format_response(response))

In [ ]:
query = 'コンピュータ'
response = vector_search(query)
print(format_response(response))

In [ ]:
query = 'コンピュータ'
response = hybrid_search(query)
print(format_response(response))

In [ ]:
es.cat.shards()

In [ ]:
es.cat.health()

In [ ]:
es.close()